In [ ]:
import pandas as pd
data = pd.read_csv(r'C:\Users\misha\Desktop\total_dataset_to_ML.csv')
data

In [ ]:
data['target'].value_counts()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from datetime import datetime
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


for i in [3,5,6,7]:
    copy_data = data.copy()
    tmp = [0, 1, 2, 3, 4, 5, 6, 7]
    tmp.remove(i)
    copy_data['target'] = copy_data['target'].replace(tmp, [0, 0, 0, 0, 0, 0, 0])
    copy_data['target'] = copy_data['target'].replace(i, 1)
    X = copy_data.drop(['target'], axis=1)
    y = copy_data['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)
    
    param_grid_svm = {
        'kernel': ['sigmoid', 'poly', 'rbf']
    }

    param_grid_knn = {
        'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12]
    }

    param_grid_dt = {
        'max_depth':  [8, 10, 12],
        'max_features': ['log2', 0.2, 0.3],
    }

    param_grid_rf = {
        'n_estimators': [100, 300],
        'max_features': ['sqrt', 0.2],
        'max_depth': [8, 10],
    }

    models = []
    models.append(('SVM', SVC(), param_grid_svm))
    models.append(('KNN', KNeighborsClassifier(), param_grid_knn))
    models.append(('DT', DecisionTreeClassifier(), param_grid_dt))
    models.append(('RF', RandomForestClassifier(), param_grid_rf))


    for name, model, parameters in models:
        name += f'_one_to_other_with_{i}_class'
        start_time = datetime.now()
        print(start_time)
        cv = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=42)
        model = GridSearchCV(estimator=model, param_grid=parameters, scoring='recall_macro', cv=cv, verbose=10, n_jobs=1)
        model.fit(X_train, y_train)
        print(name)
        print('best score: ', model.best_score_)
        print('best estimator: ', model.best_estimator_)
        pred = model.best_estimator_.predict(X_test)
        print(confusion_matrix(y_test, pred))
        print(classification_report(y_test, pred))
        print(name, datetime.now() - start_time)